In [3]:
import torch

def load_file(f):
  with open(f) as f:
    return f.read().strip().splitlines()

In [4]:
def parse_line(line):
  if line == 'noop':
    return dict(op='noop')
  else:
    [instruction, value] = line.split(' ')
    return dict(op=instruction, value=int(value))

def parse_file(content):
  return [parse_line(line) for line in content]

In [5]:
parse_file(load_file('example.txt'))

[{'op': 'noop'}, {'op': 'addx', 'value': 3}, {'op': 'addx', 'value': -5}]

In [6]:
def run_instruction (instruction, cycles):
  op = instruction['op']
  head = cycles[-1]
  if op == 'noop':
    cycles.append(head)
    return cycles
  elif op == 'addx':
    cycles.append(head)
    cycles.append(head + instruction['value'])
    return cycles

In [7]:
def run_file(file):
  content = load_file(file)
  instructions = parse_file(content)
  cycles = [1]
  for instruction in instructions:
    cycles = run_instruction(instruction, cycles)
  return torch.tensor(cycles)

In [8]:
run_file('example.txt')

tensor([ 1,  1,  1,  4,  4, -1])

In [9]:
run_file('example_long.txt')[torch.tensor([20,60,100,140,180,220]) - 1]

tensor([21, 19, 18, 21, 16, 18])

In [10]:
def get_answer(file):
  cycles = run_file(file)
  signals_index = torch.tensor(range((cycles.shape[0] - 20) // 40 + 1)) * 40 + 20
  # Adjust signals back one to fit array 0 index
  signals = cycles[signals_index - 1]
  strengths = signals_index * signals
  return strengths.sum()

In [11]:
get_answer('input.txt')

tensor(14920)

In [12]:
torch.tensor(range((250 - 20) // 40 + 1)) * 40 + 20

tensor([ 20,  60, 100, 140, 180, 220])

In [27]:
def render_crt(file):
  cycles = run_file(file)
  grid = []
  for i in range(1, cycles.shape[0]):
    point = cycles[i - 1]
    col = (i % 40)
    offset = col - point
    is_valid = offset == 0 or offset == 1 or offset == 2
    if (i > 8 and i < 12):
      print(point, col)
      print(offset, is_valid)

    if is_valid:
      grid.append(1)
    else:
      grid.append(0)
      
  return torch.tensor(grid).view(-1, 40)



In [17]:
def print_grid(grid):
  for row in grid:
    for cell in row:
      if cell == 1:
        print('#', end='')
      else:
        print('.', end='')
    print('')

In [29]:
print_grid(render_crt("input.txt"))

tensor(7) 9
tensor(2) tensor(True)
tensor(7) 10
tensor(3) tensor(False)
tensor(12) 11
tensor(-1) tensor(False)
###..#..#..##...##...##..###..#..#.####.
#..#.#..#.#..#.#..#.#..#.#..#.#..#....##
###..#..#.#....#..#.#....###..#..#...#.#
#..#.#..#.#....####.#....#..#.#..#..#..#
#..#.#..#.#..#.#..#.#..#.#..#.#..#.#....
###...##...##..#..#..##..###...##..####.
